.. _tick-locators:

# Tick Locators

When you create a figure in Toyplot, you begin by creating a :class:`canvas<toyplot.canvas.Canvas>`, add :mod:`axes<toyplot.axes>`, and add data to the axes in the form of marks.  The axes map data from its *domain* to a *range* on the canvas, generating *ticks* in domain-space with *tick labels* as an integral part of the process.  

To generate tick locations and tick labels, the axes delegate to the *tick locator* classes in the :mod:`toyplot.locator` module.  Each tick locator class is responsible for generating a collection of tick locations from the range of values in an axis domain, and there are several different classes available that implement different strategies for generating "good" tick locations.  If you don't specify any tick locators when creating axes, sensible defaults will be chosen for you.  For example:

In [1]:
import numpy
x = numpy.arange(20)
y = numpy.linspace(0, 1, len(x)) ** 2

In [2]:
import toyplot
canvas, axes, mark = toyplot.plot(x, y, width=300)

Note that the Y axis in this plot has sensible ticks that cover the full data domain $[0, 1]$, while the X axis also has sensible ticks that include "round" numbers like 20 even though the X values only cover $[0, 19]$.  In this case the algorithm for identifying "good" "round" tick values is provided by Toyplot's :class:`toyplot.locator.Extended` locator.

However, let's say that we prefer to always have ticks that include the exact minimum and maximum data domain values, and evenly divide the rest of the domain.  In this case, we can override the default choice of locator with the :class:`toyplot.locator.Basic` tick locator:

In [3]:
canvas, axes, mark = toyplot.plot(x, y, width=300)
axes.x.ticks.locator = toyplot.locator.Basic(count=5)

We can also override the default formatting string used to generate the locator labels:

In [4]:
canvas, axes, mark = toyplot.plot(x, y, width=300)
axes.x.ticks.locator = toyplot.locator.Basic(count=5, format="{:.2f}")

Anytime you use log scale axes in a plot, Toyplot automatically uses the :class:`toyplot.locator.Log` locator to provide ticks that are evenly-spaced :

In [5]:
canvas, axes, mark = toyplot.plot(x, y, xscale="log10", width=300)

If you don't like the "superscript" notation that the Log locator produces, you could replace it with your own locator and custom format:

In [6]:
canvas, axes, mark = toyplot.plot(x, y, xscale="log10", width=300)
axes.x.ticks.locator = toyplot.locator.Log(base=10, format="{base}^{exponent}")

Or even display raw tick values:

In [7]:
canvas, axes, mark = toyplot.plot(x, y, xscale="log2", width=300)
axes.x.ticks.locator = toyplot.locator.Log(base=2, format="{:.0f}")

Although you might not think of :ref:`table-axes` as needing tick locators, when you use :func:`toyplot.matrix` or :meth:`toyplot.canvas.Canvas.matrix` to visualize a matrix of values, it generates a table visualization and uses :class:`toyplot.locator.Integer` locators to generate row and column labels:

In [8]:
numpy.random.seed(1234)
canvas, table = toyplot.matrix(numpy.random.random((5, 5)), width=300)

By default the Integer locator generates a tick/label for every integer in the range $[0, N)$ ... as you visualize larger matrices, you'll find that a label for every row and column becomes crowded, in which case you can override the default `step` parameter to space-out the labels:

In [9]:
canvas, table = toyplot.matrix(numpy.random.random((50, 50)), width=400, step=5)

## Explicit Locators

For the ultimate flexibility in positioning tick locations and labels, you can use the :class:`toyplot.locator.Explicit` locator.  With it, you can specify an explicit set of labels, and a set of $[0, N)$ integer locations will be created to match.  This is particularly useful if you are working with categorical data:

In [10]:
fruits = ["Apples", "Oranges", "Kiwi", "Miracle Fruit", "Durian"]
counts = [452, 347, 67, 21, 5]

canvas, axes, mark = toyplot.bars(counts, width=400, height=300)
axes.x.ticks.locator = toyplot.locator.Explicit(labels=fruits)

Note that in the above example the implicit $[0, N)$ tick locations match the implicit $[0, N)$ X coordinates that are generated for each bar when you don't supply any X coordinates of your own.  This is by design!

You can also use Explicit locators with a list of tick locations, and a set of tick labels will be generated using a format string.  For example:

In [11]:
x = numpy.linspace(0, 2 * numpy.pi)
y = numpy.sin(x)
locations=[0, numpy.pi/2, numpy.pi, 3*numpy.pi/2, 2*numpy.pi]

canvas, axes, mark = toyplot.plot(x, y, width=500, height=300)
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, format="{:.2f}")

Finally, you can supply both locations and labels to an Explicit locator:

In [12]:
labels = ["0", u"\u03c0 / 2", u"\u03c0", u"3\u03c0 / 2", u"2\u03c0"]

canvas, axes, mark = toyplot.plot(x, y, width=500, height=300)
axes.x.ticks.locator = toyplot.locator.Explicit(locations=locations, labels=labels)

## Timestamp Locators

Toyplot now includes :class:`toyplot.locator.Timestamp` locators that can be used to provide human-consumable date-time labels when your data's domain is *timestamps* (seconds since the Unix epoch, i.e. midnight, January 1st, 1970).  As an example, let's load some real-world data containing date-time information:

In [13]:
import toyplot.data
data = toyplot.data.read_csv("commute-obd.csv")
data[:6]

timestamp,name,value,units
2014-05-01 14:08:53.587607,Status Since DTC Cleared,0011110100110,
2014-05-01 14:08:53.656972,Fuel System Status,0000,
2014-05-01 14:08:53.726403,Calculated Load Value,0.0,%
2014-05-01 14:08:53.734393,Coolant Temperature,13,C
2014-05-01 14:08:53.804349,Short Term Fuel Trim,0.0,%
2014-05-01 14:08:53.873862,Long Term Fuel Trim,-3.125,%


Our first step will be to convert the human-readable times from the file into true numeric timestamps:

In [14]:
import datetime
timestamps = [datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f") for timestamp in data["timestamp"]]
timestamps = [(timestamp - datetime.datetime(1970, 1, 1)).total_seconds() for timestamp in timestamps]
timestamps = numpy.array(timestamps)

Now, we can plot the data using the timestamps as our independent variable:

In [15]:
observations = numpy.logical_and(data["name"] == "Vehicle Speed", data["value"] != "NODATA")
x = timestamps[observations]
y = data["value"][observations]

In [16]:
canvas, axes, mark = toyplot.plot(x, y, label="Vehicle Speed", xlabel="Time", ylabel="km/h", width=600, height=300)

As you would expect, the timestamps make very unfriendly tick labels.  Let's use the timestamp locator instead:

In [17]:
canvas, axes, mark = toyplot.plot(x, y, label="Vehicle Speed", xlabel="Time", ylabel="km/h", width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp()

By default, the timestamp locator chooses "good" time intervals to cover the data domain - in this case, every five minutes.  When it does so, it also chooses a default format based on the interval - in this case, `month/day hour:minute`.  The labels are a little crowded, so let's look at different ways to clean things up.

First, we can request that the locator choose an interval that produces a specific number of ticks:

In [18]:
canvas, axes, mark = toyplot.plot(x, y, label="Vehicle Speed", xlabel="Time", ylabel="km/h", width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(count=5)

Note that in most cases the locator won't produce the exact number of ticks requested (we got lucky in this case), it will choose whichever interval produces the closest match.

Alternatively, we might choose to accept the default tick count and alter the tick format, to save space:

In [19]:
canvas, axes, mark = toyplot.plot(x, y, label="Vehicle Speed", xlabel="Time", ylabel="km/h", width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(format="{0:%H}:{0:%M}")

The individual tick values are passed to the format function as :class:`datetime.datetime` objects, so you can use any of the datetime attributes in the format string, and any of the `strftime()` percent fields, as we've done here.

Or, we might force a specific interval, if it had special meaning for our domain:

In [20]:
canvas, axes, mark = toyplot.plot(x, y, label="Vehicle Speed", xlabel="Time", ylabel="km/h", width=600, height=300)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(interval=(10, "minutes"))

Toyplot supports common intervals from seconds to millennia.

Finally, we might go in the opposite direction and show more detail in the timestamps.  This is often a case where you'll want to adjust the orientation of the labels so they don't overlap (note in the following example that we had to make the canvas larger and position the axes manually on the canvas to make room for the vertical labels - and we manually placed the x axis label to avoid overlap):

In [21]:
canvas = toyplot.Canvas(width=600, height=350)
axes = canvas.axes(bounds=(80, -80, 50, -120), label="Vehicle Speed", ylabel="km/h")
axes.plot(x, y)
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Timestamp(format="{0:%B} {0.day}, {0:%Y} {0:%H}:{0:%M}")
axes.x.ticks.labels.angle = 30
axes.x.ticks.labels.style = {"baseline-shift":0, "text-anchor":"end"}
axes.x.ticks.labels.offset = 8
canvas.text(300, 320, "Time", style={"font-weight":"bold"});

Finally note that the timestamps in your data **must be UTC**.